# PYNQ KV260 design with HLS AXI Master

NOTE: You should read the readme.md file in this repository before running this notebook.

In [1]:
from pynq import Overlay
ol = Overlay("design_1_wrapper.bit")

ol.download() 
ol.timestamp # print the time when the bit stream had been downloaded into FPGA

'2023/3/4 18:9:51 +30616'

In [2]:
from pynq import PL
#check which bit file you used to program the FPGA
PL.bitfile_name 
# check when did the FPGA had been programmed  
PL.timestamp
# check FPGA had been programed 
ol.is_loaded()

True

In [3]:
# It list all the interface and register for this FPGA design. 
# In this case, example_0 is the top design that inplemented the getCo function.
ol.ip_dict


{'axi_intc_0': {'type': 'xilinx.com:ip:axi_intc:4.1',
  'mem_id': 's_axi',
  'memtype': 'REGISTER',
  'gpio': {},
  'interrupts': {'intr': {'controller': 'axi_intc_0',
    'index': 0,
    'fullpath': 'axi_intc_0/intr'}},
  'parameters': {'C_FAMILY': 'zynquplus',
   'C_INSTANCE': 'design_1_axi_intc_0_0',
   'C_S_AXI_ADDR_WIDTH': '9',
   'C_S_AXI_DATA_WIDTH': '32',
   'C_NUM_INTR_INPUTS': '1',
   'C_NUM_SW_INTR': '0',
   'C_KIND_OF_INTR': '0xfffffffe',
   'C_KIND_OF_EDGE': '0xFFFFFFFF',
   'C_KIND_OF_LVL': '0xFFFFFFFF',
   'C_ASYNC_INTR': '0xFFFFFFFE',
   'C_NUM_SYNC_FF': '2',
   'C_ADDR_WIDTH': '32',
   'C_IVAR_RESET_VALUE': '0x0000000000000010',
   'C_ENABLE_ASYNC': '0',
   'C_HAS_IPR': '1',
   'C_HAS_SIE': '1',
   'C_HAS_CIE': '1',
   'C_HAS_IVR': '1',
   'C_HAS_ILR': '0',
   'C_IRQ_IS_LEVEL': '1',
   'C_IRQ_ACTIVE': '0x1',
   'C_DISABLE_SYNCHRONIZERS': '0',
   'C_MB_CLK_NOT_CONNECTED': '1',
   'C_HAS_FAST': '0',
   'C_EN_CASCADE_MODE': '0',
   'C_CASCADE_MASTER': '0',
   'Component_N

In [4]:
# Now we load the design and inspect the register map.
# You can also see some user defined input output, in my case are
    # IN_O_X used for the first input array
    # IN_1_X Used for the second
    # Note the interface actually create two subset,e.g. IN_O_1 and IN_0_2, Only work with the port with _1, dont know why
    # ALso the returend OUT_Corr_X
    
my_ip = ol.example_0
my_ip.register_map


RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  IN_0_1 = Register(IN_0=write-only),
  IN_0_2 = Register(IN_0=write-only),
  IN_1_1 = Register(IN_1=write-only),
  IN_1_2 = Register(IN_1=write-only),
  OUT_Corr_1 = Register(OUT_Corr=write-only),
  OUT_Corr_2 = Register(OUT_Corr=write-only)
}

In [5]:
from pynq import allocate
import numpy as np 

# Now we can allocate a puthonBuffer for our inputs and output, 
# Here tutorial uses the pythonBuffer data type, modification of python code may required if we want to use numpy type, 
IN_BUFF_0 = allocate(shape=(5,), dtype=np.double)
IN_BUFF_1 = allocate(shape=(5,), dtype=np.double)
OUT_BUFF = allocate(shape=(2,), dtype=np.double)

LIST_IN_BUFF_0 =[-5.061244639011612, 9.39905932245856, 5.521358200447387, -3.904872069362163, 17.396453218094308]
LIST_IN_BUFF_1 =[41.46545312310088, 22.992557140364028, -13.982653668826686, -7.533390612593724, -18.637688038773916]

for x in range(5):
    IN_BUFF_0[x] = LIST_IN_BUFF_0[x]
    IN_BUFF_1[x] = LIST_IN_BUFF_1[x]
   

for x in range(2):
    OUT_BUFF[x] = 0
    

In [6]:
#check the init of your buffer 
IN_BUFF_0

PynqBuffer([-5.06124464,  9.39905932,  5.5213582 , -3.90487207,
            17.39645322])

In [7]:
IN_BUFF_1

PynqBuffer([ 41.46545312,  22.99255714, -13.98265367,  -7.53339061,
            -18.63768804])

In [8]:
OUT_BUFF

PynqBuffer([0., 0.])

In [9]:
# We can inspect the address of our buffer
# the HLS interface we are using here requires us to get the addess, you can see inthe HLS code,
    # pointer had been used, the design will take the inicial addess of the array then start the process
print("IN_BUFF_0 physical address {}".format(hex(IN_BUFF_0.physical_address)))
print("IN_BUFF_1 physical address {}".format(hex(IN_BUFF_1.physical_address)))
print("OUT_BUFF physical address {}".format(hex(OUT_BUFF.physical_address)))


IN_BUFF_0 physical address 0x77339000
IN_BUFF_1 physical address 0x7733a000
OUT_BUFF physical address 0x7733b000


In [10]:
# write the address into the registers
my_ip.register_map.IN_0_1 = IN_BUFF_0.physical_address
my_ip.register_map.IN_1_1 = IN_BUFF_1.physical_address
my_ip.register_map.OUT_Corr_1 = OUT_BUFF.physical_address


# Important to start the execution,
# The incorrect Vivado block design will somehow disable the CTRL signal,
    # It means, you can see the CTRL register in the HLS,but you can not see it in python when execute the following code
my_ip.register_map.CTRL.AP_START=1


In [11]:
# check if execution had done
# if 0 , it done
my_ip.register_map.CTRL.AP_DONE


1

In [12]:
OUT_BUFF

PynqBuffer([23.35075403, 24.30427794])

In [13]:
my_ip.register_map.CTRL.AUTO_RESTART=1